# Normalizing flows for the scalar example

In [1]:
# Import required packages
import torch, math
import numpy as np
import normflows as nf

from matplotlib import pyplot as plt
from tqdm import tqdm

import sys, os, pickle
BASE_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(BASE_DIR)
from assistive_functions import *
from experiments.scalar.loss_functions import LQLossFH
from controllers.abstract import get_controller
from experiments.scalar.LTI_sys import LTI_system
from inference_algs.distributions import GibbsPosterior
from experiments.scalar.scalar_assistive_functions import load_data

[INFO] running on CPU


/Users/mahrokh/anaconda3/envs/NOC/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ****** PART 1: GENERAL ******
random_seed = 33
random_state = np.random.RandomState(random_seed)
logger = WrapLogger(None)

# ------ 1. load data ------
T = 10
S = 8
epsilon = 0.2       # PAC holds with Pr >= 1-epsilon
dist_type = 'N biased'
prior_type_b = 'Gaussian_biased_wide'

data_train, data_test, disturbance = load_data(
    dist_type=dist_type, S=S, T=T, random_seed=random_seed,
    S_test=None     # use a subset of available test data if not None
)

print(epsilon, prior_type_b, S)

# ------ 2. define the plant ------
sys = LTI_system(
    A = np.array([[0.8]]),  # num_states*num_states
    B = np.array([[0.1]]),  # num_states*num_inputs
    C = np.array([[0.3]]),  # num_outputs*num_states
    x_init = 2*np.ones((1, 1)),  # num_states*1
)

# ------ 3. define the loss ------
Q = 5*torch.eye(sys.num_states).to(device)
R = 0.003*torch.eye(sys.num_inputs).to(device)
# optimal loss bound
loss_bound = 1
# sat_bound = np.matmul(np.matmul(np.transpose(sys.x_init), Q) , sys.x_init)
sat_bound = torch.matmul(torch.matmul(torch.transpose(sys.x_init, 0, 1), Q), sys.x_init)
if loss_bound is not None:
    logger.info('[INFO] bounding the loss to ' + str(loss_bound))
lq_loss_bounded = LQLossFH(Q, R, T, loss_bound, sat_bound, logger=logger)
lq_loss_original = LQLossFH(Q, R, T, None, None, logger=logger)

# ------ 4. Gibbs temperature ------
gibbs_lambda_star = (8 * S * math.log(1/epsilon))**0.5        # lambda for Gibbs

# ------ 5. controller ------
# define a generic controller
generic_controller = get_controller(
    controller_type='Affine', sys=sys,
    # initialization_std=0.1, # for initializing REN. not important
)


0.2 Gaussian_biased_wide 8
[INFO] bounding the loss to 1


In [ ]:
# define target distribution
gibbs_posteior = GibbsPosterior(
    loss_fn=lq_loss_bounded, lambda_=gibbs_lambda_star, prior_dict, initialization_std,
    # attributes of the CL system
    controller_type, sys,
    # REN controller
    n_xi=None, l=None, x_init=None, u_init=None,
    # misc
    logger=None,
)

In [ ]:
K = 16
#torch.manual_seed(0)

# Move model on GPU if available
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')

flows = []
for i in range(K):
    flows += [nf.flows.Planar((2,))]
target = nf.distributions.TwoModes(2, 0.1)

q0 = nf.distributions.DiagGaussian(2)
nfm = nf.NormalizingFlow(q0=q0, flows=flows, p=target)
nfm.to(device)

In [ ]:

# load
file_path = os.path.join(BASE_DIR, 'experiments', 'scalar', 'saved_results')
filename = dist_type.replace(" ", "_")+'_ours_'+prior_type+'_T'+str(T)+'_S'+str(S)+'_eps'+str(int(eps_heat*10))+'.pkl'
filename = os.path.join(file_path, filename)
filehandler = open(filename, 'rb')
res_dict = pickle.load(filehandler)
filehandler.close()

res_dict['theta_grid'] = [k[0,0] for k in res_dict['theta_grid']]
res_dict['theta'] = [k[0,0] for k in res_dict['theta']]

theta_grid = - np.array(res_dict['theta_grid'])
bias_grid = - np.array(res_dict['bias_grid'])
Z_posterior = np.reshape(
    np.array(res_dict['posterior']),
    (len(theta_grid), len(bias_grid))
)
assert abs(sum(sum(Z_posterior))-1)<=1e-5


In [ ]:
# Plot target distribution
# grid_size = 200
# xx, yy = torch.meshgrid(X, torch.linspace(-3, 3, grid_size))
# z = torch.cat([xx.unsqueeze(2), yy.unsqueeze(2)], 2).view(-1, 2)
# log_prob = target.log_prob(z.to(device)).to('cpu').view(*xx.shape)
# prob = torch.exp(log_prob)

plt.figure(figsize=(10, 10))
plt.pcolormesh(bias_grid, theta_grid, Z_posterior, shading='nearest')
plt.show()

# Plot initial flow distribution
z, _ = nfm.sample(num_samples=2 ** 20)
z_np = z.to('cpu').data.numpy()
plt.figure(figsize=(10, 10))
plt.hist2d(
    z_np[:, 0].flatten(), z_np[:, 1].flatten(), 
    (len(bias_grid), len(theta_grid)), 
    range=[[bias_grid[0], bias_grid[-1]], [theta_grid[0], theta_grid[-1]]]
)
plt.show()

In [ ]:
# Train model
max_iter = 20000
num_samples = 2 * 20
anneal_iter = 10000
annealing = True
show_iter = 2000


loss_hist = np.array([])

optimizer = torch.optim.Adam(nfm.parameters(), lr=1e-3, weight_decay=1e-4)
for it in tqdm(range(max_iter)):
    optimizer.zero_grad()
    if annealing:
        loss = nfm.reverse_kld(num_samples, beta=np.min([1., 0.01 + it / anneal_iter]))
    else:
        loss = nfm.reverse_kld(num_samples)
    loss.backward()
    optimizer.step()
    
    loss_hist = np.append(loss_hist, loss.to('cpu').data.numpy())
    
    # Plot learned distribution
    if (it + 1) % show_iter == 0:
        torch.cuda.manual_seed(0)
        z, _ = nfm.sample(num_samples=2 ** 20)
        z_np = z.to('cpu').data.numpy()
        
        plt.figure(figsize=(10, 10))
        plt.hist2d(z_np[:, 0].flatten(), z_np[:, 1].flatten(), (grid_size, grid_size), range=[[-3, 3], [-3, 3]])
        plt.show()

plt.figure(figsize=(10, 10))
plt.plot(loss_hist, label='loss')
plt.legend()
plt.show()

In [ ]:
# Plot learned distribution
z, _ = nfm.sample(num_samples=2 ** 20)
z_np = z.to('cpu').data.numpy()
plt.figure(figsize=(10, 10))
plt.hist2d(z_np[:, 0].flatten(), z_np[:, 1].flatten(), (grid_size, grid_size), range=[[-3, 3], [-3, 3]])
plt.show()